In [ ]:
import os
import numpy as np

from matplotlib import pyplot as plt

# Dataset
fname = "jena_climate_2009_2016.csv"

# Open file
f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header)-1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i,:] = values

temp = float_data[:,1]
plt.plot(range(len(temp)), temp)
plt.show()

In [ ]:
# Relevant variables
step = 6               # one data point per hour
delay = 144             # targets will be 24h in the future
num_days = 5            # Number of considered days
lookback = delay*num_days          # observations go back 5 days

In [ ]:
# Normalizing the data
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


In [ ]:
def generator(data, lookback, delay, min_index, max_index,
    shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
    
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))
    
    for j, row in enumerate(rows):
        indices = range(rows[j] - lookback, rows[j], step)
        samples[j] = data[indices]
        targets[j] = data[rows[j] + delay][1]
    
#Return sends a specified value back to its caller whereas 
# Yield can produce a sequence of values. We should use yield 
# when we want to iterate over a sequence, but don’t want 
# to store the entire sequence in memory.

    yield samples, targets

In [ ]:
# Preparing the training, validation and test generators
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
    lookback=lookback, delay=delay, min_index=0,
    max_index=200000, shuffle=True, step=step, batch_size=batch_size)
val_gen = generator(float_data,
    lookback=lookback, delay=delay, min_index=200001,
    max_index=300000, step=step, batch_size=batch_size)
test_gen = generator(float_data,
    lookback=lookback, delay=delay, min_index=300001, max_index=None,
    step=step, batch_size=batch_size)

val_steps = (300000 - 200001 - lookback)
test_steps = (len(float_data) - 300001 - lookback)

In [13]:
#Machine Learning Approach
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

model = Sequential()
model.add(layers.Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen, steps_per_epoch=500,
                        epochs = 20, validation_data=val_gen,
                        validation_steps=val_steps)

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss)+1)

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


ValueError: Could not interpret optimizer identifier: <tensorflow.python.keras.optimizers.RMSprop object at 0x00000199B46296A0>